# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [144]:
//DONE

val housing = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("data/housing.csv")

housing = [longitude: double, latitude: double ... 8 more fields]


[longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [145]:
// DONE

housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [146]:
// DONE

housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [147]:
// DONE

housing.take(5)

Array([-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY], [-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY], [-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY], [-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY], [-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY])

Print the number of records with population more than 10000.

In [148]:
// DONE

housing.filter("population > 10000").count()

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [149]:
// DONE

housing.select("housing_median_age", "total_rooms", "median_house_value", "population").describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [150]:
// DONE

import org.apache.spark.sql.functions._

housing.select(max("housing_median_age"), min("total_rooms"), avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [151]:
// DONE

housing.groupBy("ocean_proximity").count().sort(desc("count")).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [152]:
// DONE

housing.groupBy("ocean_proximity").agg(avg("median_house_value").alias("avg_value")).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [153]:
// DONE

housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value "+
           "FROM df "+
           "GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [154]:
// DONE

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().
             setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).
             setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

va = vecAssembler_9d38ca5c4373
housingAttrs = [longitude: double, latitude: double ... 9 more fields]


[longitude: double, latitude: double ... 9 more fields]

In [155]:
// DONE

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head()

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


coeff = 


1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [156]:
// DONE

val housingCol1 = housing.withColumn("rooms_per_household", housing("total_rooms")/housing("households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", housing("total_bedrooms")/housing("total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household", housing("population")/housing("households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1 = [longitude: double, latitude: double ... 9 more fields]
housingCol2 = [longitude: double, latitude: double ... 10 more fields]
housingExtra = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [157]:
// DONE

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")

renamedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [158]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel = label
colCat = ocean_proximity
colNum = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [159]:
// DONE

for (c <- colNum) {
    val count = renamedHousing.filter(renamedHousing(c).isNull || renamedHousing(c)==="").count
    println(s"Missing values in ${c}: ${count}")
}

Missing values in longitude: 0
Missing values in latitude: 0
Missing values in housing_median_age: 0
Missing values in total_rooms: 0
Missing values in total_bedrooms: 207
Missing values in population: 0
Missing values in households: 0
Missing values in median_income: 0
Missing values in rooms_per_household: 0
Missing values in bedrooms_per_room: 207
Missing values in population_per_household: 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [160]:
// DONE

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").
                            setInputCols(Array("total_bedrooms", "bedrooms_per_room")).
                            setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))

val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)
imputedHousing.select("*").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129

imputer = imputer_84fa32685200
imputedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [161]:
// DONE

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("feature_scaling")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("feature_scaling").setOutputCol("scaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|     feature_scaling|     scaled_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

va = vecAssembler_de8dd9367555
featuredHousing = [longitude: double, latitude: double ... 12 more fields]
scaler = stdScal_ef877fb41fee
scaledHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [162]:
// DONE

renamedHousing.select(countDistinct("ocean_proximity")).show

+-------------------------------+
|count(DISTINCT ocean_proximity)|
+-------------------------------+
|                              5|
+-------------------------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [163]:
// DONE

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("indexed_ocean_proximity")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|indexed_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.2

indexer = strIdx_aac9e4564288
idxHousing = [longitude: double, latitude: double ... 12 more fields]


[longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [164]:
indexer.fit(renamedHousing).labels

Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [165]:
// DONE

import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().
                  setInputCols(Array("indexed_ocean_proximity")).
                  setOutputCols(Array("encoded_ocean_proximity"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|indexed_ocean_proximity|encoded_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21

encoder = oneHotEncoder_9e9a69638e45
ohHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [166]:
// DONE

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))

val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|     feature_scaling|     scaled_features|indexed_ocean_proximity|encoded_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14

numPipeline = pipeline_791833128a5c
catPipeline = pipeline_e3d4220deb8b
pipeline = pipeline_b078a5fb54df
newHousing = [longitude: double, latitude: double ... 15 more fields]


[longitude: double, latitude: double ... 15 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [167]:
// DONE

val va2 = new VectorAssembler().setInputCols(Array("scaled_features", "encoded_ocean_proximity")).setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows



va2 = vecAssembler_60df383831cf
dataset = [features: vector, label: double]


[features: vector, label: double]

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [168]:
// DONE

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet = [features: vector, label: double]
testSet = [features: vector, label: double]


[features: vector, label: double]

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [169]:
// DONE

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setFeaturesCol("features").setLabelCol("label")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-55543.807375002834,-55708.08572321854,14185.035315963396,5672.066615265683,5543.550570999544,-44307.05985699013,38244.52787353796,79038.98355945593,6717.0416342088965,16409.101030186663,279.7851876007071,-176727.5759550153,-211047.5227586806,-171794.45519115386,-180556.02697966163], Intercept: -2268290.3484457093
RMSE: 67879.25669743802


lr = linReg_d57671afd5a4
lrModel = linReg_d57671afd5a4
trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@d2aacd2


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@d2aacd2

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [170]:
// DONE

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|143258.71128097177|103600.0|[-62.040445150874...|
|181261.07677898603|107000.0|[-61.980550915813...|
| 161789.0565961697| 70200.0|[-61.980550915813...|
|124775.72381554684|105900.0|[-61.975559729558...|
|171791.14444586495| 90600.0|[-61.970568543303...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67977.09538900755


predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_c6dfaed3d27b
rmse = 67977.09538900755


67977.09538900755

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [171]:
// DONE

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 170060.8091844814|103600.0|[-62.040445150874...|
| 170060.8091844814|107000.0|[-61.980550915813...|
| 170060.8091844814| 70200.0|[-61.980550915813...|
|150757.70609318998|105900.0|[-61.975559729558...|
|150757.70609318998| 90600.0|[-61.970568543303...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67566.4055666165


dt = dtr_46679fd2de32
dtModel = DecisionTreeRegressionModel (uid=dtr_46679fd2de32) of depth 5 with 63 nodes
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_5e188bd29b50
rmse = 67566.4055666165


67566.4055666165

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [172]:
// DONE

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|174567.60750612564|103600.0|[-62.040445150874...|
| 181243.2257858337|107000.0|[-61.980550915813...|
|155845.51897178224| 70200.0|[-61.980550915813...|
|151612.57502330764|105900.0|[-61.975559729558...|
|157162.29180927633| 90600.0|[-61.970568543303...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 64946.451661245024


rf = rfr_acc01bc38f6e
rfModel = RandomForestRegressionModel (uid=rfr_acc01bc38f6e) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_a0e50d84757c
rmse = 64946.451661245024


64946.451661245024

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [173]:
// DONE

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|119320.29768940082|103600.0|[-62.040445150874...|
|101640.02926073727|107000.0|[-61.980550915813...|
|109129.24749163276| 70200.0|[-61.980550915813...|
| 94431.81743735791|105900.0|[-61.975559729558...|
| 93742.17652206896| 90600.0|[-61.970568543303...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 55421.71635693072


gb = gbtr_997fc1f60904
gbModel = GBTRegressionModel (uid=gbtr_997fc1f60904) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_652768f2face
rmse = 55421.71635693072


55421.71635693072

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [ ]:
// DONE

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
   .addGrid((rfModel).numTrees, Array(1, 5, 10))
   .addGrid(rfModel.maxDepth, Array(5, 10, 15))
   .build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val cv = new CrossValidator()
    .setEstimator(rf)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [ ]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType, StringType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "Input column 1")
    final val inputCol2 = new Param[String](this, "inputCol2", "Input column 2")
    final val outputCol = new Param[String](this, "outputCol", "Output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    val idx1 = schema.fieldIndex($(inputCol1))
    val field1 = schema.fields(idx1)
    val idx2 = schema.fieldIndex($(inputCol2))
    val field2 = schema.fields(idx2)
    if (field1.dataType != StringType && field2.dataType != StringType) {
      throw new Exception(s"Input type ${field1.dataType} or ${field2.dataType} did not match input type StringType")
    }
    schema.add(StructField($(outputCol), DoubleType, false)) 
  }
}



Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [ ]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.{StructField, StructType, DoubleType, StringType}
import org.apache.spark.sql.functions.{col, udf}

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap) = {
    defaultCopy(extra)
  }
    
    override def transformSchema(schema: StructType): StructType = {
    val idx1 = schema.fieldIndex($(inputCol1))
    val field1 = schema.fields(idx1)
    val idx2 = schema.fieldIndex($(inputCol2))
    val field2 = schema.fields(idx2)
    if (field1.dataType != StringType && field2.dataType != StringType) {
      throw new Exception(s"Input type ${field1.dataType} or ${field2.dataType} did not match input type StringType")
    }
    schema.add(StructField($(outputCol), DoubleType, false)) 
  }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        import dataset.sparkSession.implicits._
     dataset.select(col("*"),(dataset($(inputCol1))/dataset($(inputCol2))).as($(outputCol)))
  }
}

Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [ ]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

In [ ]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

### Load the data

In [ ]:
val credit_cards = spark.read.format("csv").option("header","true").option("inferSchema","true").load("data/ccdefault.csv")

In [ ]:
//rename label as target
val labeled_cards = credit_cards.withColumnRenamed("DEFAULT","LABEL")

### Explanatory analysis

In [ ]:
//in the schema there is no string input, therefore we don't need any StringIndexer.
labeled_cards.printSchema()

In [ ]:
//no missing values in the dataset, therefore we don't need to use an Imputer.
for (c <- labeled_cards.columns) {
    print(c + " ")
    println(labeled_cards.select(c).where(c+" IS NULL").count())
}

In [ ]:
labeled_cards.count()

In [ ]:
%%dataframe
labeled_cards

PAY: repayment status is: -1 = pay duly; 1 = payment delay for one month etc.
BILL_AMT: amount of bill statement (NT dollar)
PAY_AMT: amount of previous payment (NT dollar)

In [ ]:
//shows dataset information
labeled_cards.select("sex", "marriage", "age", "education","limit_bal").describe().show()

In [ ]:
//shows dataset further information
labeled_cards.groupBy("label").count().withColumn("percentage",col("count")/30000).show()
labeled_cards.groupBy("sex").count().withColumn("percentage",col("count")/30000).show()
labeled_cards.groupBy("marriage").count.withColumn("percentage",col("count")/30000).show()
print("Distinct ages: " + labeled_cards.select("age").distinct.count +"\n\n")
labeled_cards.groupBy("education").count.withColumn("percentage",col("count")/30000).show()
print("Distinct credits: "+ labeled_cards.select("limit_bal").distinct.count)

//IMPORTANT: the dataset is highly unbalanced. We need to study F1-score and AUC

### Data cleaning

In [ ]:
//fixes the first PAY column and drops the ID column since not useful.
val new_labeled_cards = labeled_cards.withColumnRenamed("PAY_0", "PAY_1").drop("id")

In [ ]:
new_labeled_cards.printSchema()

### Data augmentation: calculate the ratio between bill and limit
In order to compare different people, we do not have to consider tha absolute value of payments but the ratio between payments and balance limit.

In [ ]:
val newdf1 = new_labeled_cards.withColumn("PayLimitRatio1", new_labeled_cards("BILL_AMT1") / new_labeled_cards("LIMIT_BAL"))
val newdf2 = newdf1.withColumn("PayLimitRatio2", newdf1("BILL_AMT2") / newdf1("LIMIT_BAL"))
val newdf3 = newdf2.withColumn("PayLimitRatio3", newdf2("BILL_AMT3") / newdf2("LIMIT_BAL"))
val newdf4 = newdf3.withColumn("PayLimitRatio4", newdf3("BILL_AMT4") / newdf3("LIMIT_BAL"))
val newdf5 = newdf4.withColumn("PayLimitRatio5", newdf4("BILL_AMT5") / newdf4("LIMIT_BAL"))
val augmented_df = newdf5.withColumn("PayLimitRatio6", newdf5("BILL_AMT6") / newdf5("LIMIT_BAL"))

Now, we want to separate the numerical attributes from the categorical attribute (sex,education,marriage) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors.

In [ ]:
// label columns
val colLabel = "LABEL"

// categorical columns
val colCat1 = "EDUCATION"
val colCat2 = "SEX"
val colCat3 = "MARRIAGE"

// numerical columns
val colNum = augmented_df.columns.filter(_ != colLabel).filter(_ != colCat1).filter(_ != colCat2).filter(_ != colCat3)

### Scaling
With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the cards data: the total number of limit_bal ranges from  10000 to 1000000, while the age only range from 21 to 79. 

To get all attributes to have the same scale we use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we use the `StandardScaler` Estimator. To use `StandardScaler`, we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vector.

In [ ]:
val vectorAssembler = new VectorAssembler().setInputCols(colNum).setOutputCol("feature_scaling")
val featured_cards = vectorAssembler.transform(augmented_df)

val standardScaler = new StandardScaler().setInputCol("feature_scaling").setOutputCol("scaled_features")
val scaled_cards = scaler.fit(featured_cards).transform(featured_cards)

scaled_cards.show(5)

Now, use VectorAssembler to put all attributes of the final dataset cards into a big vector, and call the new column features.

In [ ]:
//Concatenate numerical attributes and categorical attributes to create a unique dataset

val new_vectorAssembler = new VectorAssembler().setInputCols(Array("scaled_features", colCat1, colCat2, colCat3)).setOutputCol("features")
val dataset = new_vectorAssembler.transform(scaled_cards).select("features", "label")

dataset.show(5)

### Models
Here we going to make four different classification models:

Logistic regression
Decision tree 
Random forest
But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (trainSet) with 80% of the whole data, and test dataset (testSet) with 20% of it.

In [ ]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8,0.2))

### Logistic Regression

In [140]:
import org.apache.spark.ml.classification.LogisticRegression

// train the model
val cclr = new LogisticRegression().setFeaturesCol("features").setLabelCol("label")
val cclrModel = cclr.fit(trainSet)
val ccdtPredictions = cclrModel.transform(testSet)
val cctrainingSummary = cclrModel.summary
ccdtPredictions.select("prediction", "rawPrediction", "label", "features").show(5)

println(s"Coefficients: " + cclrModel.coefficients + s", Intercept: " + cclrModel.intercept)

+----------+--------------------+-----+--------------------+
|prediction|       rawPrediction|label|            features|
+----------+--------------------+-----+--------------------+
|       0.0|[1.00253811574144...|    1|(23,[0,1,2,3,4,5,...|
|       0.0|[2.94939935328374...|    0|(23,[0,1,2,3,4,5,...|
|       0.0|[0.61545739275624...|    1|(23,[0,1,2,3,4,5,...|
|       0.0|[1.01509086454154...|    0|(23,[0,1,2,3,4,5,...|
|       0.0|[2.76677618828046...|    0|(23,[0,1,2,3,4,5,...|
+----------+--------------------+-----+--------------------+
only showing top 5 rows

Coefficients: [-0.11914018864471992,0.06894728861820569,0.653905173432875,0.09457482194003618,0.05524008554089173,0.04847319010977171,0.033829776093898534,0.015331652209844664,-0.3959610273840587,0.07695059301674045,0.24801350636587843,-0.05967593593596619,0.015780081984488457,0.04026783511959722,-0.19706788574932416,-0.2687730309530074,-0.014137457469266165,-0.04335119803101174,-0.0488921257427708,-0.03732360050231556,-0.

cclr = logreg_7ab86808955e
cclrModel = LogisticRegressionModel: uid = logreg_7ab86808955e, numClasses = 2, numFeatures = 23
ccdtPredictions = [features: vector, label: int ... 3 more fields]
cctrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@3fa9065b


org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@3fa9065b

In [141]:
import org.apache.spark.ml.classification.LogisticRegression
val cctrainingSummary = cclrModel.binarySummary
val ccobjectiveHistory = cctrainingSummary.objectiveHistory
val roc = cctrainingSummary.roc
roc.show()
println(s"areaUnderROC: ${cctrainingSummary.areaUnderROC}")

+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|0.003455976180348788| 0.0328515111695138|
|0.007018290089323692| 0.0653275764970903|
| 0.01015525308379413|0.09930542519241599|
|  0.0141429179072735| 0.1302797071522433|
| 0.01781156954487452| 0.1623803266378825|
|0.020735857082092725|0.19710906701708278|
|0.025202041684389623|0.22639384268819224|
|0.029934070608251807|0.25474000375445843|
| 0.03514461931093152|0.28139665853200674|
|0.039610803913228416| 0.3106814342031162|
| 0.04514036580178647|0.33621175145485266|
| 0.05098894087622288|0.36061573118077717|
| 0.05752871118672905|0.38257931293410924|
| 0.06513185878349638| 0.4007884362680683|
| 0.07273500638026371| 0.4189975596020274|
| 0.08097618034878774|0.43495400788436267|
| 0.08958953636750319| 0.4497841186408861|
| 0.09846873670778392|0.46386333771353483|
| 0.10761378136962994| 0.4768162192603717|
+----------

cctrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@3fa9065b
ccobjectiveHistory = Array(0.5278136968325416, 0.4835616642923949, 0.47993527239101386, 0.47905791028380723, 0.4741284663968141, 0.4702398568830792, 0.46642430986665195, 0.4661842011113599, 0.46562146855846515, 0.46552504847597764, 0.4653687900793237, 0.4652568219008303, 0.4651306387180005, 0.46505023472623047, 0.4650091488747398, 0.4649891699850269, 0.46495081776126707, 0.46490933929177397, 0.4648454571360966, 0.4647687246457498, 0.46473810913206576, 0.46471352834238133, 0.4646960670804229, 0.4646665159031471, 0.46466018966660066,...


Array(0.5278136968325416, 0.4835616642923949, 0.47993527239101386, 0.47905791028380723, 0.4741284663968141, 0.4702398568830792, 0.46642430986665195, 0.4661842011113599, 0.46562146855846515, 0.46552504847597764, 0.4653687900793237, 0.4652568219008303, 0.4651306387180005, 0.46505023472623047, 0.4650091488747398, 0.4649891699850269, 0.46495081776126707, 0.46490933929177397, 0.4648454571360966, 0.4647687246457498, 0.46473810913206576, 0.46471352834238133, 0.4646960670804229, 0.4646665159031471, 0.46466018966660066,...

In [142]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val cclrParamGrid = new ParamGridBuilder()
    .addGrid(cclr.regParam, Array(0,0.001, 0.01, 0.1)).addGrid(cclr.elasticNetParam, Array(0, 0.0001, 0.001, 0.01))
    .build()

val cclrEvaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction")//.setPredictionCol("prediction").setMetricName("accuracy")

val cclrCv = new CrossValidator()
    .setEstimator(cclr)
    .setEvaluator(cclrEvaluator)
    .setEstimatorParamMaps(cclrParamGrid)
    .setNumFolds(4)

val cclrCvModel = cclrCv.fit(dataset)

val cclrPredictions = cclrCvModel.transform(testSet)
cclrPredictions.select("prediction", "label", "features").show(5)

val cclrAuc = cclrEvaluator.evaluate(cclrPredictions)
println(s"Area Under ROC (AUC) on test data = "+cclrAuc)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    1|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.6091752847679098


cclrParamGrid = 


Array({
	logreg_7ab86808955e-elasticNetParam: 0.0,
	logreg_7ab86808955e-regParam: 0.0
}, {
	logreg_7ab86808955e-elasticNetParam: 1.0E-4,
	logreg_7ab86808955e-regParam: 0.0
}, {
	logreg_7ab86808955e-elasticNetParam: 0.001,
	logreg_7ab86808955e-regParam: 0.0
}, {
	logreg_7ab86808955e-elasticNetParam: 0.01,
	logreg_7ab86808955e-regParam: 0.0
}, {
	logreg_7ab86808955e-elasticNetParam: 0.0,
	logreg_7ab86808955e-regParam: 0.001
}, {
	logreg_7ab86808955e-elasticNetParam: 1.0E-4,
	logreg_7ab86808955e-regParam: 0.001
}, {
	logreg_7ab86808955e-elasticNetParam: 0.001,
	logre...


### Decision Tree 

In [143]:

import org.apache.spark.ml.classification.DecisionTreeClassifier

val ccdt = new DecisionTreeClassifier().setFeaturesCol("features").setLabelCol("label")
val ccdtModel = ccdt.fit(trainSet)

val ccdtPredictions1 = ccdtModel.transform(testSet)
ccdtPredictions1.select("prediction", "rawPrediction", "label", "features").show(5)

val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction")//.setMetricName("accuracy")
val accuracy = evaluator.evaluate(ccdtPredictions1)


val ccdtParamGrid = new ParamGridBuilder().addGrid(ccdt.maxBins, Array(20,40,80)).addGrid(ccdt.maxDepth, Array(5,10,20)).build()

val ccdtCv = new CrossValidator().setEstimator(ccdt).setEvaluator(evaluator).setEstimatorParamMaps(ccdtParamGrid).setNumFolds(4)

val ccdtCvModel = ccdtCv.fit(dataset)

val ccdtPredictions = ccdtCvModel.transform(testSet)
cclrPredictions.select("prediction", "label", "features").show(5)

val ccdtAuc = evaluator.evaluate(cclrPredictions)
println(s"Area Under ROC (AUC) on test data = "+ccdtAuc)

+----------+----------------+-----+--------------------+
|prediction|   rawPrediction|label|            features|
+----------+----------------+-----+--------------------+
|       0.0|[16960.0,2827.0]|    1|(23,[0,1,2,3,4,5,...|
|       0.0|[16960.0,2827.0]|    0|(23,[0,1,2,3,4,5,...|
|       0.0|   [826.0,462.0]|    1|(23,[0,1,2,3,4,5,...|
|       0.0|[16960.0,2827.0]|    0|(23,[0,1,2,3,4,5,...|
|       0.0|[16960.0,2827.0]|    0|(23,[0,1,2,3,4,5,...|
+----------+----------------+-----+--------------------+
only showing top 5 rows

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    1|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    1|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.6091752847679098


ccdt = dtc_26f2adb2985d
ccdtModel = DecisionTreeClassificationModel (uid=dtc_26f2adb2985d) of depth 5 with 39 nodes
ccdtPredictions1 = [features: vector, label: int ... 3 more fields]
evaluator = binEval_d8033d44a842
accuracy = 0.6486693727694607
ccdtParamGrid = 


Array({
	dtc_26f2adb2985d-maxBins: 20,
	dtc_26f2adb2985d-maxDepth: 5
}, {
	dtc_26f2adb2985d-maxBins: 20,
	dtc_26f2adb2985d-maxDepth: 10
}, {
	dtc_26f2adb2985d-maxBins: 20,
	dtc_26f2adb2985d-maxDept...


### Random Forest

In [96]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}


val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features")

// train the model
val rfMdel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfMdel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val rfEvaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction")//.setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
///

val rfParamGrid = new ParamGridBuilder().addGrid(rf.numTrees, Array(2,5,15)).addGrid(rf.maxDepth, Array(5,10,20)).addGrid(rf.maxBins, Array(20,40,80)).build()

val rfCv = new CrossValidator().setEstimator(rf).setEvaluator(rfEvaluator).setEstimatorParamMaps(rfParamGrid).setNumFolds(4)

val rfModel = rfCv.fit(dataset)

val rfPredictions = rfModel.transform(testSet)
cclrPredictions.select("prediction", "label", "features").show(5)

val rfAuc = evaluator.evaluate(rfPredictions)
println(s"Area Under ROC (AUC) on test data = $rfAuc")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.6591848595108146


rf = rfc_d9dccda8df6b
rfMdel = RandomForestClassificationModel (uid=rfc_d9dccda8df6b) with 20 trees
predictions = [features: vector, label: int ... 3 more fields]
rfEvaluator = binEval_7c93f28f6ae3
accuracy = 0.6570140994542146
rfParamGrid = 


Array({
	rfc_d9dccda8df6b-maxBins: 20,
	rfc_d9dccda8df6b-maxDepth...
